# Part 3: creating our own objects

In [1]:
from utils import get_fruits
apples, bananas, oranges = get_fruits()

Up to here we've been treating these Fruits and Baskets as black boxes. And, quite often, that is how you'll use them. You will take some object that someone else has made, read the instructions and use it. 

But soon we're going to want to make our own. So, let's start by looking inside the objects, and seeing how they are made. 

-----

### Taking a look under the hood 
Remember that we used a `Fruit` class to create the apples? This is how the `Fruit` class was actually created (slightly simplified, we'll add stuff later): 

In [2]:
class Fruit:
    def __init__(self, name, price_per_unit, days_until_expired, nr_units):
        self.name = name
        self.nr_units = nr_units
        self.price_per_unit = price_per_unit
        self.days_until_expired = days_until_expired
        
    # there will be more stuff here (methods), I'm hiding them for now for simplicity 

In [3]:
# we can then use this Fruit to create fruits, as we've seen before 
apple = Fruit(name='royal gala', price_per_unit=2, days_until_expired=20, nr_units=4)

We immediately notice a few things. 
1. The arguments are in some weird `__init__` thing
2. There is a lot of the word `self` going around

We'll get to these, one at the time. 

----

### Understanding \_\_init__

The information we wanted to store (price, nr_units...) is all going into this `__init__` thing. 

You probably remember that in functions we pass arguments with `def(something, something_else)`, and that was the local scope inside the function, right? 

Well, in classes you define the attributes (remember them?) of the class using this `__init__`.  There is nothing clever about it, it's one of those things you just have to memorize. 

In classes, you define the attributes in the `__init__`. Repeat 20 times until it sinks in. 

In [4]:
# let's do a few examples 
class Person: 
    def __init__(self, name, age):
        self.name = name
        self.age = age

class Virus:
    def __init__(self, name, ro0, discovery_date):
        self.name = name
        self.ro0 = ro0
        self.discovery_date = discovery_date

Ok, so the \_\_init\_\_ is where we pass "information we need the class to know", right? 

Yes, but not just that. Consider the following Dog: 

In [5]:
class Dog:
    def __init__(self, name, weight):  # <-- we never said anything about mammals... 
        self.name = name
        self.weight=weight
        self.family = "mammal"          # <-- and yet there it is! 

You may be thinking "hang on, why didn't we pass the family, and yet it gets defined as mammal?" 

That is because all dogs are mammals. It would not make sense to ask the user to pass the family every time they want to create a new dog, and yet it may be useful (for instance if a Zoo object wants to figure out how many mammals it has).

Here is another thing you'll see quite often:

In [6]:
class Human:
    def __init__(self, name, date_of_birth): 
        self.name = name 
        self.date_of_birth = date_of_birth
        self.date_of_death = None             # <--- wait what?

In this case, we don't know the date of death at the time when we are creating Humans. We may be logging these humans in some system and, by the time they die, we'll update this value. 

But when we instantiate (fancy word to say "use the class to create instances"), we create the class arguments that we expect we'll need. So, we need the `date_of_death`, even if we don't have a value for it yet, so we set it to None. 

Now let's instantiate a human, called "frank".

In [7]:
frank = Human('Frank Smith', date_of_birth='1974-04-12')

What is Frank Smith's date of death?

In [8]:
print(frank.date_of_death)

None


Great! And now... let's kill him! 

In [9]:
frank.date_of_death = '2020-05-27'   # bye frank! 

Great, what's his date of death now? 

In [10]:
print(frank.date_of_death)

2020-05-27


Great. So, we can update a method after the instance has been created. Which is very useful. 

You may be wondering: _"is this just because it was set to None?"_
The answer is no, we can also update attributes. 

Let's be even more evil, and change Frank's name to _"Frankenstein Smith"_

Remember, this is name before we change it: 

In [11]:
frank.name 

'Frank Smith'

And after we update his name: 

In [12]:
frank.name = "Frankenstein Smith"
print(frank.name)

Frankenstein Smith


Here is a related example from our good old friend, the `Basket`: 

In [13]:
class Basket:
    def __init__(self):
        self.products = []   # <-- empty list! 
        
    # there will be more stuff here (methods), I'm hiding them for now for simplicity 

In the case of a basket, we instantiate new baskets without any products, so we create them with an empty list. The methods will then take care of updating this list, but given that we will need it, we start it as empty with the \_\_init\_\_. 

----

### Understanding self

Ok, let's deal with `self`, the elephant in the room. It would be `self`ish not to (I'm so sorry.)

Before we jump into it, consider the following example:

In [14]:
# Create a class for Person 
class Person:
    def __init__(self, name, age):
        self.name = name 
        self.age = age
        
amy = Person(name='Amy Malkovich', age=24)

print(amy.age)

24


How does it know that amy has an age? Simple: the Person class had an age. When `amy` got instantiated, everywhere the word `self` used to be became `amy`. 

Capiche? `amy.age` exists because the Person class had a `self.age`. 

![](https://i.imgflip.com/3wdi4x.jpg)

In the `__init__`, we said "we're going to pass two things when we create a new Person: a `name`, and an `age`". 

When we create `amy` we passed her a `name` and an `age`. When we created an instance of a Person `self` for that person got replaced with the instance variable name. 

In [15]:
# Create a class for Person 
class Person:
    def __init__(self, name, age):
        self.name = name   # <-- we don't know whether will be amy or john, self is a placeholder
        self.age = age
        
amy = Person(name='Amy Malkovich', age=24)
john = Person(name='John Malkovich', age=57)

# What happens if we use attributes that weren't in the class that generated amy? 
# amy does not have a height, because the class Person does not have a self.height
try: 
    amy.height
except AttributeError as e: 
    print('This failed! Here is the error: {0}'.format(e))

This failed! Here is the error: 'Person' object has no attribute 'height'


### Understanding instance methods 

We know from our good old friend the `Basket` that classes can be used for more than just storing information. The basket had this cool ability to sum the price of all items in it, remember? 

This was done with a function that we put in the method. But before we do any of that, let's illustrate why that is useful, and pretend methods don't exist. 

Here is what the world looks like before we introduce methods: 

In [16]:
# Create a class for Person 
class Person:
    def __init__(self, name, age):
        self.name = name 
        self.age = age

# now let's create a function that takes an instance of a person, and ages them by one 
def celebrate_birthday_simple(some_person):
    some_person.age += 1  # increase the person's age by 1 
    return some_person 

# now let's create a person: 
mark = Person(name='Mark Smith', age=22)
# print their name and age: 
print('{0} is {1} years old'.format(mark.name, mark.age))

# now let's use the function to make mark one year older 
mark = celebrate_birthday_simple(mark)  # <--- pay special attention to this! 

# print their name and age after going through the function: 
print('{0} is {1} years old (after going through the function)'.format(mark.name, mark.age))

Mark Smith is 22 years old
Mark Smith is 23 years old (after going through the function)


So you will have noticed this slightly weird thing: 

```
# "mark equals celebrate birthday of mark"
mark = celebrate_birthday_simple(mark)
```

This is kind of dirty and requires us to go around overwriting variables. Also, this `celebrate_birthday_simple` function is clearly very specific to Person, as it even knows that they have an `age` property. 

```
def celebrate_birthday_simple(some_person):
    some_person.age += 1  # <--- this function is really specific to Person! 
    return some_person 
```

What we want is to say that this "celebrate birthday" thing is a method of the Person Class (it belongs to the Person Class). So we'll put it in the Person class: 

In [17]:
class Person:
    def __init__(self, name, age):
        self.name = name 
        self.age = age
    
    def celebrate_birthday(self):
        self.age += 1  # increase the person's age by 1 

Remember what we said about the "self" stuff. It just refers to the "thing" (in this case the person) that is getting its attribute updated. When we are creating the Person we don't know if it will be mark, john or mary, but we do know that it will be... the self. 

It's a bit non-intuitive at first, but just take the word `self` very literally: 
- `self.name = name` means `update the name on myself to the name that gets passed to me`
- `self.age += 1` means `update the age by one on myself`

Look at the two versions next to each other:

In [18]:
class Person:
    def __init__(self, name, age):
        self.name = name 
        self.age = age    
    def celebrate_birthday(self):   # <-- here defined as an instance method on your class
        self.age += 1  
        # no need to return self, because we've already altered it!

def celebrate_birthday_simple(some_person):  # <-- here defined outside the class 
    some_person.age += 1   # <-- notice that "some_person" takes the place of "self"!
    return some_person 

Key thing to notice: wherever it says "some_person" on the outside function, we say "self" in the instance method. 

So, when you are creating a `Fruit` class, at some point you will create some `apple` with it. Whenever it would make sense to have `apple`, you write `self`. 

It is a very common mistake for even experienced python users to forget the pass self as an argument in an instance method. This is an example of that mistake: 

In [19]:
class Basket:
    def __init__(self):
        self.products = []   
        
    def add_item_to_basket(item):  # <-- oh no! I forgot self! 
        self.products.append(item)  # <-- wait, what do you mean "self"? 
        return self 

You will notice that it allowed me to get this far. Let's see what happens when I create a basket and try to use it: 

In [20]:
my_basket = Basket()
# note: don't worry about the try/except, it's just that we're doing something that will fail
try:    # 
    my_basket.add_item_to_basket(apple)
except TypeError as e: 
    print('Oh no! This failed! Here is the error: \n   {0}'.format(e))

Oh no! This failed! Here is the error: 
   add_item_to_basket() takes 1 positional argument but 2 were given


Well... that's not a very helpful error! The real issue is that it expects 1 argument, but what got passed implicitly was `(self, item)`. Given that it had only been told to expect `item`, this error will show up. 

You'll see this error again in your life. Oh yes, you'll see it again. 

Correct implementation of our basket: 

In [21]:
class Basket:
    def __init__(self):
        self.products = []   
        
    def add_item_to_basket(self, item):  # <-- this time with self 
        self.products.append(item)
        
my_basket = Basket()
my_basket.add_item_to_basket(apple)

Ok, you got this far, bravo! 

Let's recapitulate. I want to go back to the Person example and show you the two implementations of how to celebrate their birthday (using a function or using an instance method) side by side. 

Remember, these are two different ways to do the same thing:

In [1]:
def celebrate_birthday_simple(some_person):  # <-- normal function 
    some_person.age += 1   
    return some_person                  

class Person:
    def __init__(self, name, age):
        self.name = name 
        self.age = age    
    def celebrate_birthday(self):   # <-- using an instance method
        self.age += 1  
        # no need to return, because we've already altered self! 

So, whenever you see this: 
- `jonh.celebrate_birthday()`

Remember that it is the same as `celebrate_birthday(jonh)`, but the `john` is already passed as `self`. 

-----

#### Calling instance methods from within the class

Sometimes, you will want to call a method using another method. As an example, consider the following implementation of the basket:

In [23]:
class Basket:
    def __init__(self):
        self.content = []

    def add_item(self, item):
        self.content.append(item)

    def check_total_price(self):
        total_price = 0
        for item in self.content:
            total_price += item.calculate_price()
        print('The total price is {0}'.format(total_price))
        
    def describe_every_item(self):
        for item in self.content:
            # print some info about the item 
            print('- {0} {1} (total price {2})'.format(
                item.nr_units, item.name, item.calculate_price()))

    def examine_basket(self):
        self.check_total_price()
        self.describe_every_item()

The `examine_basket` method does two things: 
1. checks the total price 
2. describes every item 

So, it wants to call the two other metods. 

The `examine_basket` method does the equivalent of this: 
```
my_basket.check_total_price()
my_basket.describe_every_item()
```

Because it is in the class, we use `self.check_total_price()` instead of `check_total_price(<something>)`

In [24]:
# create a basket 
my_basket = Basket()

# this is also a common pattern, adding things from a list, or spreadsheet, etc
for item in [apples, bananas, oranges]: 
    my_basket.add_item(item)
    
my_basket.examine_basket()

The total price is 28
- 10 apples (total price 10)
- 6 bananas (total price 12)
- 2 oranges (total price 6)


-----

Ok, quick summary of the learnings so far: 

- Classes (e.g. Fruit) are used to make instances of the class (e.g. `apples`) 
- Attributes are pieces of information that belong to the class (e.g. `apples.price`) 
- Instance methods are functions that belong to the class (e.g. `appples.add_item_to_basket`)
- The `__init__` is where the class attributes are initially defined. 
- The `self` is used when defining the class. It is a "placeholder" that you would replace with the instance name if you were doing it as an external function. 

Ok, now go on and do the rest of the exercises! Good luck! 

----